In [1]:
# scraping the data from the web
import pandas as pd
import numpy as np

from urllib.request import urlopen

from bs4 import BeautifulSoup
page_list=[
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4343;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4347;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4344;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=5845;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4342;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4788;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4341;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4346;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4787;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4345;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=5843;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=4340;type=trophy",
          "http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?id=117;team=5143;type=trophy"]

names = [       
    "Chennai Super Kings","Deccan Chargers","Delhi Daredevils","Gujarat Lions","Kings XI Punjab",
    "Kochi Tuskers Kerala","Kolkata Knight Riders","Mumbai Indians","Pune Warriors","Rajasthan Royals",
    "Rising Pune Supergiants","Royal Challengers Bangalore","Sunrisers Hyderabad"
    ]
ext = ".csv"
k=0
for i in page_list:
    page = urlopen(i)
    soup = BeautifulSoup(page)
    right_table=soup.find('table', class_='engineTable')
    l = []
    th=right_table.find_all("th")
    head=[hd.text for hd in th]
    for tr in right_table.findAll("tr"):
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    df=pd.DataFrame(l)
    df.columns=head
#     print("head",head)
#     print(df.head())
    df.to_csv(names[k]+ext,index=False)
    k+=1
    


#(important note)after manual cleaning, concatenating the files




# from glob import glob
# filenames=glob("*.csv")
# headers=["player_name","matches_played","innings_batted","not_outs","runs_scored","highest_inns_score",
#          "batting_average","balls_faced","batting_strike_rate","hundreds_scored","fifties_scored",
#          "duckes_scored","boundary_fours","boundary_sixes"]   

# dfs=[pd.read_csv(x,names=headers) for x in filenames]
# combined=pd.concat(dfs)
# combined.to_csv("all_teams_batting.csv",index=False)




/usr/lib/python3/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [2]:
# df=df.iloc[2:]
# print(df.head())

from glob import glob
filenames=glob("*.csv")
dfs=[]
headers=["player_name","matches_played","innings_batted","not_outs","runs_scored","highest_inns_score",
         "batting_average","balls_faced","batting_strike_rate","hundreds_scored","fifties_scored",
         "duckes_scored","boundary_fours","boundary_sixes"]
for i in filenames:
    if(i[:-4] in names):
        df1=pd.read_csv(i)
        df1=df1.iloc[1:]
        if 'Span' in df1.columns:
            df1.drop(columns=['Span'],inplace=True)
        df1.columns=headers        
        dfs.append(df1)

combined=pd.concat(dfs)
combined.to_csv("all_teams_batting.csv",index=False)
combined.shape

(953, 14)

In [3]:
df=pd.read_csv("all_teams_batting.csv")
df.head()

,player_name,matches_played,innings_batted,not_outs,runs_scored,highest_inns_score,batting_average,balls_faced,batting_strike_rate,hundreds_scored,fifties_scored,duckes_scored,boundary_fours,boundary_sixes
0,VR Aaron,6.0,2,1,4,4,4.00,10,40.00,0,0,0,0,0
1,KJ Abbott,5.0,3,2,13,12*,13.00,8,162.50,0,0,1,0,1
2,YA Abdulla,11.0,1,1,0,0*,-,1,0.00,0,0,0,0,0
3,L Ablish,3.0,1,1,0,0*,-,2,0.00,0,0,0,0,0
4,AD Nath,1.0,1,0,9,9,9.00,13,69.23,0,0,0,1,0


In [4]:
i = df[df.runs_scored=='-'].index #get index of the players who hasn't played a single innigns

In [5]:
df.drop(i,inplace=True) #remove the players who hasn't played a single innigns

In [6]:
df.dtypes
df.shape

(876, 14)

In [7]:
#type casting
df['runs_scored']=df['runs_scored'].astype('int64')
df['balls_faced']=df['balls_faced'].astype('int64')
df['innings_batted']=df['innings_batted'].astype('int64')
df['not_outs']=df['not_outs'].astype('int64')

In [15]:
#calculating number of times a player is out (required for batting average)
df['times_out']=df['innings_batted']-df['not_outs'] 

In [16]:
#grouping multiple instances of a player into single instance
res = df.groupby('player_name').agg({'runs_scored':'sum','balls_faced':'sum','times_out':'sum'}).reset_index()

In [17]:
#calculating batting average and strike rate
res['batting_avg']=(res['runs_scored']/res['times_out']).round(2)
res['strike_rate']=((res['runs_scored']/res['balls_faced'])*100).round(2)

In [18]:
#replace infinity values with Nan
res['batting_avg']=res['batting_avg'].replace(np.inf,np.nan)

In [21]:
res.dropna(inplace=True)
res.to_csv('stats_batting_2.csv',index=None)